## Purpose

The purpose of this notebook is to display a working example of how to use Google's universal sentence encoder to compare two different strings. The general idea is to apply some basic NLP techniques with Spacy in order to increase the weights of the 'important' aspects of a sentence, then apply the sentence encoder to get a vector representation of the sentances. These sentances are later compared.

In general, there will be one news article that is inputted. This will be compared to a dataframe of songs that have already been processed by the encoder. The one with the best cosine similarity will be selected.

In [8]:
import numpy as np
import pandas as pd
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
from newspaper import Article
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import EnglishStemmer
import sentencepiece
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import cross_val_score
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
def embed(text):
    print('Start')
    print('Starting embeddings...')
    #embed_US = hub.Module("universal_sentence")
    embed_US = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")
    embeddings = embed_US(text)
    print('Extracting embeddings...')
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        embd = sess.run(embeddings)
    dim_vector = ['Dim_{}'.format(i) for i in range(embd.shape[1])]
    df_return = pd.DataFrame(embd, columns = dim_vector)
    return df_return

In [10]:
# #First we want to import the particular article that we want
article = Article('https://www.cbc.ca/news/world/cocaine-bust-philadelphia-ship-1.5180447')
article.download()
article.parse()
article_text=article.text

In [12]:
#now we import all of the song Lyrics (this should probably be done in another python script)
df_songs = pd.DataFrame(pd.read_csv('songdata.csv'))


In [13]:
df_songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [14]:
df_songs['artist'].unique()

array(['ABBA', 'Ace Of Base', 'Adam Sandler', 'Adele', 'Aerosmith',
       'Air Supply', 'Aiza Seguerra', 'Alabama', 'Alan Parsons Project',
       'Aled Jones', 'Alice Cooper', 'Alice In Chains', 'Alison Krauss',
       'Allman Brothers Band', 'Alphaville', 'America', 'Amy Grant',
       'Andrea Bocelli', 'Andy Williams', 'Annie', 'Ariana Grande',
       'Ariel Rivera', 'Arlo Guthrie', 'Arrogant Worms', 'Avril Lavigne',
       'Backstreet Boys', 'Barbie', 'Barbra Streisand', 'Beach Boys',
       'The Beatles', 'Beautiful South', 'Beauty And The Beast',
       'Bee Gees', 'Bette Midler', 'Bill Withers', 'Billie Holiday',
       'Billy Joel', 'Bing Crosby', 'Black Sabbath', 'Blur', 'Bob Dylan',
       'Bob Marley', 'Bob Rivers', 'Bob Seger', 'Bon Jovi', 'Boney M.',
       'Bonnie Raitt', 'Bosson', 'Bread', 'Britney Spears',
       'Bruce Springsteen', 'Bruno Mars', 'Bryan White', 'Cake',
       'Carly Simon', 'Carol Banawa', 'Carpenters', 'Cat Stevens',
       'Celine Dion', 'Chaka Khan

In [14]:
#political_artists = ['eminem-d12','eminem']#,'bob-marley' 'bob-marley-the-wailers','beyonce-knowles']
#df_songs_political = df_songs[df_songs['artist'].isin(political_artists)]
df_songs_political.head()


,index,song,year,artist,genre,lyrics
283031,283031,fight-music,2013,eminem-d12,Hip-Hop,[Chorus: Eminem]\nThis kinda music\nUse it and...
283032,283032,keep-talkin,2008,eminem-d12,Hip-Hop,"Yeah, Detroit, motherfucka\nDJ Green Lantern, ..."
283033,283033,i-ll-shit-on-you,2008,eminem-d12,Hip-Hop,"I'll shit on you, da da, da da, da da\nI'll sh..."
313003,313003,people-make-me,2009,eminem,Hip-Hop,NaN
313004,313004,my-darling,2009,eminem,Hip-Hop,Ya look\nIf I were to rap about the crap that'...


In [ ]:
df_songs_political=df_songs_political.dropna(subset=['lyrics'])
df_songs_political['lyrics'] = df_songs_political['lyrics'].apply(lambda x: x.replace('\n',' '))


In [ ]:
df_songs_politcal_lyrics= list(df_songs_political.iloc[:,5])

## Here we begin to implement some of the NLP

Implement stemming after

In [ ]:
#This cell removes stop words and weights the description to focus on Nouns, Adjectives, and Verbs.
def nlp_weighting(input_list):
    print('Start')
    nlp = spacy.load('en')
    newtext = []

    for doc in input_list:
        nlpdoc=nlp(doc)
        tempDoc=''
        for token in nlpdoc:
            if token.is_stop == False:
                tempDoc = tempDoc + ' ' + str(token.lemma_)
                if token.pos_ == 'NOUN':
                    #We triple the strength of Nouns
                    tempDoc = tempDoc + ' ' + str(token.lemma_)
                    tempDoc = tempDoc + ' ' + str(token.lemma_)
                elif  token.pos_ == 'ADJ':
                    #We double the strength fo Adjectives
                    tempDoc = tempDoc + ' ' + str(token.lemma_)
                elif token.pos_ == 'VERB':
                    #We double the strength of Verbs
                    tempDoc = tempDoc + ' ' + str(token.lemma_)
                    
        #Here we have a hard cutoff at 2100 characters. THis is because there were memory issues with the encoding otherwise
        if len(tempDoc) > 2100:
            tempDoc = tempDoc[0:2100]
        if len(tempDoc) < 110:
            tempDoc =''

        newtext.append(tempDoc)
        
    print('Returned')
        
    return(newtext)
    
    

In [18]:
df_songs_politcal_lyrics = nlp_weighting(df_songs_politcal_lyrics)

Start
Returned


Note below we have the version where we are taking TFIDF Weights. In reality, this would be harder to implement. In practice, we will use a pretrained model that will be able to return a vector to compare similarities. Also, we will want to restrict the size of our data in order to make comparisons feasible.

Implement embedding below

In [19]:
len(df_songs_politcal_lyrics)

581

In [22]:
df_songs_political_lyrics_embed = embed(df_songs_politcal_lyrics)

Start
Starting embeddings...
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Extracting embeddings...


In [23]:
df_songs_political_lyrics_embed.head()

,Dim_0,Dim_1,Dim_2,Dim_3,Dim_4,Dim_5,Dim_6,Dim_7,Dim_8,Dim_9,Dim_10,Dim_11,Dim_12,Dim_13,Dim_14,Dim_15,Dim_16,Dim_17,Dim_18,Dim_19,Dim_20,Dim_21,Dim_22,Dim_23,Dim_24,Dim_25,Dim_26,Dim_27,Dim_28,Dim_29,Dim_30,Dim_31,Dim_32,Dim_33,Dim_34,Dim_35,Dim_36,Dim_37,Dim_38,Dim_39,...,Dim_472,Dim_473,Dim_474,Dim_475,Dim_476,Dim_477,Dim_478,Dim_479,Dim_480,Dim_481,Dim_482,Dim_483,Dim_484,Dim_485,Dim_486,Dim_487,Dim_488,Dim_489,Dim_490,Dim_491,Dim_492,Dim_493,Dim_494,Dim_495,Dim_496,Dim_497,Dim_498,Dim_499,Dim_500,Dim_501,Dim_502,Dim_503,Dim_504,Dim_505,Dim_506,Dim_507,Dim_508,Dim_509,Dim_510,Dim_511
0,-0.043546,-0.046377,-0.014012,-0.033482,-0.055708,-0.016451,-0.028052,-0.029006,-0.002859,-0.051293,0.053667,0.023362,-0.051412,0.055936,0.038279,0.051678,-0.044512,-0.012593,0.051773,0.012877,-0.052250,-0.047550,0.046867,0.041088,0.047814,0.003871,-0.038804,0.033263,0.046392,0.035826,-0.031177,0.054402,0.052534,-0.039143,0.014950,0.049223,-0.053642,-0.018879,0.039539,0.033272,...,-0.055726,0.055944,-0.054300,0.053138,-0.052542,0.055765,-0.055936,0.012610,0.048227,-0.039743,0.014616,-0.003097,-0.048924,-0.030053,-0.055942,0.053398,-0.054866,0.004922,-0.048198,0.044411,0.023653,0.055739,0.054967,-0.053261,0.055898,-0.013618,0.051285,0.042338,0.037476,0.026317,-0.049983,-0.003013,0.055946,-0.055643,0.053186,-0.055921,0.049664,-0.022258,-0.055845,0.055201
1,-0.003506,0.043842,0.021781,-0.046783,-0.051274,0.025513,-0.050620,-0.036133,-0.050709,-0.045455,0.051265,-0.016655,-0.016872,0.051274,0.049925,0.051271,-0.043107,-0.051254,-0.018499,0.049012,-0.051197,-0.051272,0.011642,-0.013668,0.051172,0.028110,-0.011781,0.010968,-0.050718,0.050289,0.001409,0.001439,-0.015662,-0.000218,-0.011160,0.051249,-0.051252,-0.000600,-0.022180,-0.015449,...,-0.050532,0.051274,-0.051005,0.042976,-0.051159,0.051255,-0.051258,0.001217,0.051201,-0.050498,0.023989,-0.047935,0.045304,-0.031570,-0.051272,0.051206,-0.051272,0.050735,-0.019595,0.043849,-0.005618,0.051185,0.051273,-0.005908,0.051274,-0.050580,0.043078,0.049279,0.035611,-0.040589,-0.050921,0.013184,0.051274,-0.051274,0.050398,-0.051274,0.047869,0.050948,-0.051272,0.051257
2,0.002697,-0.051373,0.022568,-0.050184,-0.053002,-0.049322,-0.049312,-0.027400,-0.042388,-0.052793,0.052982,0.043161,-0.052176,0.041648,0.049289,0.053044,0.002314,-0.037124,0.044187,0.006503,-0.051850,-0.034948,0.044539,0.006391,0.050408,-0.007173,-0.051856,-0.038889,-0.049801,0.025422,0.001417,0.000536,0.017925,0.050527,0.002679,0.053064,-0.051955,0.000199,0.051017,0.047810,...,-0.048877,0.053073,-0.052839,0.039596,-0.052893,0.050257,-0.053079,0.040647,0.023479,-0.046963,-0.047918,-0.051605,0.024196,0.010640,-0.053081,0.048377,-0.053000,-0.025640,-0.023859,0.031428,0.025570,0.053069,0.052906,0.041750,0.053041,0.024757,0.051872,0.048532,0.001577,-0.018895,-0.053058,0.031322,0.053080,-0.053073,0.051952,-0.053045,0.052621,0.052023,-0.052959,0.052749
3,-0.044234,0.016732,0.039142,0.014995,-0.051512,-0.010867,-0.046778,-0.048023,-0.049232,-0.049120,0.051501,0.040127,0.031349,0.051513,0.049891,0.051491,-0.049659,-0.051510,-0.004021,0.051088,-0.048757,-0.051510,-0.046646,-0.050969,-0.031060,0.045033,-0.032078,0.043303,-0.049398,0.048492,0.033718,0.044021,0.030418,-0.033932,0.049070,-0.007646,-0.043801,-0.049927,0.050049,-0.039664,...,-0.049972,0.051513,-0.048868,0.049629,-0.051484,0.051481,-0.051513,-0.051437,0.045637,0.025813,0.017431,0.049011,0.043400,-0.048180,-0.051503,0.050818,-0.051513,0.036421,-0.006904,0.051502,-0.000419,0.050034,0.051491,-0.051422,0.051513,-0.051227,-0.015385,0.008416,0.047469,-0.048844,-0.049808,0.048959,0.051513,-0.051513,0.045779,-0.051513,0.034426,-0.004095,-0.051497,0.050780
4,-0.020392,0.052606,0.012197,-0.040473,-0.054818,-0.006115,-0.034589,-0.028364,-0.050616,-0.038945,0.054817,0.038759,-0.053308,0.054816,0.053928,0.053771,-0.053332,-0.052260,0.028108,0.050143,-0.052209,-0.054399,0.018219,-0.033890,0.035585,0.025232,-0.032538,0.017320,-0.050271,-0.020892,0.024738,0.022261,0.029353,-0.010272,0.040482,0.052811,-0

### Below we apply the NLP to the Article

We will use the functions from above to do this

In [28]:
article_text = nlp_weighting([article_text])

Start
Returned


In [30]:
article_text_embed = embed([article_text[0],'temp']).iloc[0,:]

Start
Starting embeddings...
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Extracting embeddings...


In [50]:
max_cos = 0
max_col = ''
for i in range(len(df_songs_political_lyrics_embed)):
    temp_cos = cosine_similarity([article_text_embed],[df_songs_political_lyrics_embed.iloc[i]])
    if temp_cos > max_cos:
        max_cos = temp_cos
        max_col = df_songs_political.iloc[i]
print(max_col)

index                                                313122
song                                             by-my-side
year                                                   2007
artist                                               eminem
genre                                               Hip-Hop
lyrics    OH! Stat Quo Here we go Come on, come on You r...
Name: 313122, dtype: object


In [16]:
try_me = pd.read_csv('Lyric_embeddings.csv')

In [17]:
try_me.head()

,Unnamed: 0,Dim_0,Dim_1,Dim_2,Dim_3,Dim_4,Dim_5,Dim_6,Dim_7,Dim_8,...,Dim_502,Dim_503,Dim_504,Dim_505,Dim_506,Dim_507,Dim_508,Dim_509,Dim_510,Dim_511
0,0,-0.064043,0.002711,0.044418,0.047437,0.017836,-0.064006,-0.017170,-0.022429,0.005674,...,0.062882,0.031975,0.058427,-0.063681,0.039541,-0.053158,0.059598,-0.057905,-0.062222,-0.012399
1,1,-0.036305,0.051495,-0.017416,0.050525,-0.051399,0.051384,-0.049624,-0.002074,-0.004920,...,0.051541,0.028690,0.051544,-0.051544,0.051535,-0.051544,0.030857,-0.050966,-0.051543,-0.050538
2,2,-0.052767,0.053203,0.050693,0.041100,-0.053927,-0.011343,-0.004317,-0.015641,-0.046941,...,0.053843,0.013186,0.053929,-0.053929,0.013442,-0.053927,0.009755,-0.042066,-0.053851,0.013447
3,3,-0.008967,0.052989,0.052803,-0.050184,-0.052186,0.052781,-0.038071,-0.012371,-0.034460,...,0.052505,0.027601,0.052968,-0.052934,0.018361,-0.052993,0.044106,-0.010870,-0.052896,0.052350
4,4,-0.012667,0.052570,0.052357,-0.051105,-0.052522,0.051983,-0.035997,-0.033544,-0.021200,...,0.041530,-0.013303,0.052434,-0.052553,0.036652,-0.052569,0.050773,0.050676,-0.052543,0.052299
